<img style="float: left; margin: 30px 15px 15px 15px;" src="https://oci02.img.iteso.mx/Identidades-De-Instancia/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg" width="500" height="250" /> 
    
    
# <font color='navy'> Backtest

<font color='black'>

- Luis Fernando Márquez Bañuelos

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ta

sns.set_theme()

In [2]:
data = pd.read_csv('aapl_5m_train.csv').dropna()

In [3]:
rsi_indicator = ta.momentum.RSIIndicator(close=data.Close, window=43)

data['rsi'] = rsi_indicator.rsi()
data['buy_signal'] = data['rsi'] < 29
data['sell_signal'] = data['rsi'] > 67

data = data.dropna()
data.head()

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,rsi,buy_signal,sell_signal
42,42,1609783200,0,2021-01-04 18:00:00,128.410003,128.600006,128.255996,128.320007,1250689.0,33.585730,False,False
43,43,1609783500,0,2021-01-04 18:05:00,128.330001,128.570007,128.050003,128.565002,1371017.0,35.751666,False,False
44,44,1609783800,0,2021-01-04 18:10:00,128.559997,128.889892,128.510498,128.779998,1313608.0,37.580593,False,False
45,45,1609784100,0,2021-01-04 18:15:00,128.774993,128.889999,128.619995,128.755004,1276775.0,37.453696,False,False
46,46,1609784400,0,2021-01-04 18:20:00,128.750106,129.070007,128.720001,129.050003,1210041.0,39.905733,False,False


In [4]:
from dataclasses import dataclass

@dataclass
class Position:
    """ A cool representation of a position
    """
    ticker: str
    n_shares: int
    price: float
    sl: float
    tp: float
    time: str

In [5]:
COM: float = 0.125 / 100
BORROW_RATE: float = 0.25 / 100

# DOF
STOP_LOSS: float = 0.1
TAKE_PROFIT: float = 0.1
N_SHARES: int = 150

capital: float = 1_000_000
active_long_positions: list[Position] = []

In [6]:
COM: float = 0.125 / 100
BORROW_RATE: float = 0.25 / 100

# DOF
STOP_LOSS: float = 0.05517069347465185
TAKE_PROFIT: float = 0.05649834752141918
N_SHARES: int = 369

capital: float = 1_000_000
active_long_positions: list[Position] = []

In [7]:
for i, row in data.iterrows():
    # -- LONG -- #
    # Check active orders
    for position in active_long_positions.copy():
        # Stop loss or take profit check
        if row.Close > position.tp or row.Close < position.sl:
            # Add profits / losses to capital
            capital += row.Close * position.n_shares * (1 - COM)
            # Remove position from active position
            active_long_positions.remove(position)

    # Check Signal
    if row.buy_signal:
        cost = row.Close * N_SHARES * (1 + COM)

        # Do we have enough cash?
        if capital > cost:
            # Discount cash
            capital -= cost
            # Add position to portfolio
            pos = Position(ticker='AAPL', n_shares = N_SHARES, price = row.Close,
                           sl = row.Close * (1 - STOP_LOSS), tp = row.Close*(1 + TAKE_PROFIT),
                           time = row.Datetime)
            active_long_positions.append(pos)

# At the end of the backtesting, we sholud close all active positions
capital += row.Close * len(active_long_positions) * N_SHARES * (1 - COM)
active_long_positions = []

In [8]:
capital

1019000.2772282623